In [4]:
import pandas as pd
import datetime

import progressbar
import time
import math
from elasticsearch import Elasticsearch
from pandas.io.json import json_normalize

def to_date(s):
    dt = datetime.datetime.fromtimestamp(s / 1e3)
    return dt.strftime("%Y-%m-%d")

#给定阈值
yz_hao, yz_tao= float(1.5),float(1.2)
yz_xin, yz_old= float(2), float(1.5) 
yz_wen,yz_zho,yz_pi = float(1.2),float(1.5),float(2)


yz_hao_xin_wen = yz_hao*yz_xin*yz_wen
yz_hao_xin_zho = yz_hao*yz_xin*yz_zho
yz_hao_xin_pi = yz_hao*yz_xin*yz_pi
yz_hao_old_wen = yz_hao*yz_old*yz_wen
yz_hao_old_zho = yz_hao*yz_old*yz_zho
yz_hao_old_pi = yz_hao*yz_old*yz_pi
yz_tao_xin_wen = yz_tao*yz_xin*yz_wen
yz_tao_xin_zho = yz_tao*yz_xin*yz_zho
yz_tao_xin_pi = yz_tao*yz_xin*yz_pi
yz_tao_old_wen = yz_tao*yz_old*yz_wen
yz_tao_old_zho = yz_tao*yz_old*yz_zho
yz_tao_old_pi = yz_tao*yz_old*yz_pi











#门店所有id
dsl_id = {
        'query': {'bool': {'must': [
                                    {'terms': {'order_type': ['vip_scan']}},
                                    {'query_string': {'query': 'skutotal_price != 0'}},
                                    {'terms': {'order_status': ['confirmed',
                                                                'done',
                                                                'fulfilled']}},
                                    
                                    ]}},
        "aggs": {
            "my_buckets": {
                "composite": {
                    'size': 9999999,
                    "sources": [
                        {"门店": {"terms": {"field": 'shop_id'}}}
                        ]
                }
            }
        }
    }
es_id = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
res_id = es_id.search(index='es_order_table', doc_type='order', body=dsl_id,
                _source=False, explain=False, track_scores=False, track_total_hits=False, size=0,
                search_type='query_then_fetch')
df_id = json_normalize(res_id['aggregations']['my_buckets']['buckets'])
df_id.columns = map(lambda x: x.split('key.')[-1] if x.startswith('key.') else x, list(df_id.columns))

order_avg,order_max,order_std,order_yssd=[],[],[],[]
bar = progressbar

for i in bar.progressbar(range(len(df_id))):
    time.sleep(0.1)
   # bar.update(i)
    
    shop_list=[]
    shop_list.append(int(df_id.iloc[i]['门店']))
    dsl_sj = {
        'query': {'bool': {'must': [
                                    {'terms': {'order_type': ['vip_scan']}},
                                    {'terms': {'shop_id': shop_list}},
                                    {'query_string': {'query': 'skutotal_price != 0'}},
                                    {'terms': {'order_status': ['confirmed',
                                                                'done',
                                                                'fulfilled']}},
                                    ]}},
            "aggs": {
                "my_buckets": {
                    "composite": {
                        'size': 9999999,
                        "sources": [
                            {"单价": {"terms": {"field": 'skutotal_price'}}},
                            {"订单": {"terms": {"field": 'order_id'}}}
                            ]
                    }
                }
            }
    }
    es_sj = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
    res_sj = es_sj.search(index='es_order_table', doc_type='order', body=dsl_sj,
                        _source=False, explain=False, track_scores=False, track_total_hits=False, size=0,
                        search_type='query_then_fetch')
    df_sj = json_normalize(res_sj['aggregations']['my_buckets']['buckets'])
    df_sj.columns = map(lambda x: x.split('key.')[-1] if x.startswith('key.') else x, list(df_sj.columns))
    #list化所有订单
    
    shop_order_1,shop_order_2,shop_order_3 = [],[],[]

    for i in range(len(df_sj)):
        if df_sj.iloc[i]['单价']<1:   #过小订单
            continue
        else:
            shop_order_1.append(df_sj.iloc[i]['单价'])

    #除去过小值后的订单价格列表   shop_order_1 除去小订单


    large_number = pd.Series(shop_order_1).quantile(0.9)  #80%价格以上的整数订单
    large_number_large = pd.Series(shop_order_1).quantile(0.95) #90以上价格且大于1500的
    #搞出两个较大值

    for i in range(len(shop_order_1)):
        if shop_order_1[i] > large_number:
            if int(shop_order_1[i])/100 == int(shop_order_1[i]/100):
                shop_order_3.append(shop_order_1[i])
            elif shop_order_1[i] > large_number_large and shop_order_1[i] > 1500:       #对手机店很不友好
                shop_order_3.append(shop_order_1[i])
        else:
            shop_order_2.append(shop_order_1[i])
            
    #shop_order_2 正常订单
    #shop_order_3 疑似套现订单

    if shop_order_2 == []:
        shop_order_2.append(0)
    if shop_order_3==[]:
        shop_order_3.append(0)
        
    order_avg.append(pd.Series(shop_order_2).mean())
    order_max.append(pd.Series(shop_order_2).max())
    order_std.append(pd.Series(shop_order_2).std())
    order_yssd.append(shop_order_3)

df_id['平均值'] = order_avg
df_id['最大值'] = order_max
df_id['标准差'] = order_std
df_id['疑似刷单价格'] = order_yssd


df_id = df_id.fillna(0)


tao_or_not,new_or_not,bodong_degree = [],[],[]

for i in bar.progressbar(range(len(df_id))):   #判断taoxian or not
    time.sleep(0.1)
    
    if df_id.iloc[i]['疑似刷单价格'] != '[0]':
        tao_or_not.append('是')
    else:
        tao_or_not.append('否')
        
        
        
        
dis_cof=[]        
for i in bar.progressbar(range(len(df_id))):                                          #设定方差分类标准
    time.sleep(0.1)
    
    dis_cof.append(df_id.iloc[i]['标准差']/df_id.iloc[i]['平均值'])

low_number = pd.Series(dis_cof).quantile(0.2)
hig_number = pd.Series(dis_cof).quantile(0.8)




first_day,last_day=[],[]

for i in bar.progressbar(range(len(df_id))):                                        #应该可以加速，查询一次就行，不会加两列
    time.sleep(0.1)
    
    if df_id.iloc[i]['标准差']/df_id.iloc[i]['平均值'] > hig_number:             #设立波动分类标准
        bodong_degree.append('波动')
    elif df_id.iloc[i]['标准差']/df_id.iloc[i]['平均值'] < low_number:
        bodong_degree.append('稳定')
    else:
        bodong_degree.append('普通')
    
    shop_list=[]
    shop_list.append(int(df_id.iloc[i]['门店']))

    dsl_list = {
                'query': {'bool': {'must': [
                                            {'terms': {'order_type': ['vip_scan']}},
                                            {'terms': {'shop_id': shop_list}},
                                            {'query_string': {'query': 'skutotal_price != 0'}},
                                            {'terms': {'order_status': ['confirmed',
                                                                        'done',
                                                                        'fulfilled']}},
                                            ]}},
                "aggs": {
                    "my_buckets": {
                        "composite": {
                            'size': 9999999,
                            "sources": [
                                {"日期": {"date_histogram": {"field": "paytime", "interval": "1d"}}},
                               # {"单价": {"terms": {"field": 'skutotal_price'}}},
                                {"门店": {"terms": {"field": 'shop_id'}}}

                                ]
                        }
                    }
                }
            }
    es_list = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
    res_list = es_list.search(index='es_order_table', doc_type='order', body=dsl_list,
                        _source=False, explain=False, track_scores=False, track_total_hits=False, size=0,
                        search_type='query_then_fetch')
    df_list = json_normalize(res_list['aggregations']['my_buckets']['buckets'])
    df_list.columns = map(lambda x: x.split('key.')[-1] if x.startswith('key.') else x, list(df_list.columns))
    df_list['日期'] = df_list['日期'].apply(to_date)

    first_day.append(df_list['日期'].min())
    last_day.append(df_list['日期'].max())

    if datetime.datetime.strptime(last_day[i],"%Y-%m-%d")-datetime.datetime.strptime(first_day[i],"%Y-%m-%d") < datetime.timedelta(7) or df_id['doc_count'][i]<10:
        new_or_not.append('新')
    else:
        new_or_not.append('旧')
        #新店旧店
        
df_id['是否套现'] = tao_or_not #是否
df_id['新店旧店'] = new_or_not #新旧
df_id['波动程度'] = bodong_degree #稳定，普通，波动

shop_shreshold=[]


for i in bar.progressbar(range(len(df_id))):
    time.sleep(0.1)
    
    if df_id.iloc[i]['是否套现'] == '否' and df_id.iloc[i]['新店旧店'] == '新' and df_id.iloc[i]['波动程度']=='稳定':
        shop_shreshold.append(500*math.ceil(yz_hao_xin_wen*df_id.iloc[i]['最大值']/500+0.01))
    elif df_id.iloc[i]['是否套现'] == '否' and df_id.iloc[i]['新店旧店'] == '新' and df_id.iloc[i]['波动程度']=='普通':
        shop_shreshold.append(500*math.ceil(yz_hao_xin_zho*df_id.iloc[i]['最大值']/500+0.01))
    elif df_id.iloc[i]['是否套现'] == '否' and df_id.iloc[i]['新店旧店'] == '新' and df_id.iloc[i]['波动程度']=='波动':
        shop_shreshold.append(500*math.ceil(yz_hao_xin_pi*df_id.iloc[i]['最大值']/500+0.01))
    elif df_id.iloc[i]['是否套现'] == '否' and df_id.iloc[i]['新店旧店'] == '旧' and df_id.iloc[i]['波动程度']=='稳定':
        shop_shreshold.append(500*math.ceil(yz_hao_old_wen*df_id.iloc[i]['最大值']/500+0.01))
    elif df_id.iloc[i]['是否套现'] == '否' and df_id.iloc[i]['新店旧店'] == '旧' and df_id.iloc[i]['波动程度']=='普通':
        shop_shreshold.append(500*math.ceil(yz_hao_old_zho*df_id.iloc[i]['最大值']/500+0.01))
    elif df_id.iloc[i]['是否套现'] == '否' and df_id.iloc[i]['新店旧店'] == '旧' and df_id.iloc[i]['波动程度']=='波动':
        shop_shreshold.append(500*math.ceil(yz_hao_old_pi*df_id.iloc[i]['最大值']/500+0.01))
    elif df_id.iloc[i]['是否套现'] == '是' and df_id.iloc[i]['新店旧店'] == '新' and df_id.iloc[i]['波动程度']=='稳定':
        shop_shreshold.append(500*math.ceil(yz_tao_xin_wen*df_id.iloc[i]['最大值']/500+0.01))
    elif df_id.iloc[i]['是否套现'] == '是' and df_id.iloc[i]['新店旧店'] == '新' and df_id.iloc[i]['波动程度']=='普通':
        shop_shreshold.append(500*math.ceil(yz_tao_xin_zho*df_id.iloc[i]['最大值']/500+0.01))
    elif df_id.iloc[i]['是否套现'] == '是' and df_id.iloc[i]['新店旧店'] == '新' and df_id.iloc[i]['波动程度']=='波动':
        shop_shreshold.append(500*math.ceil(yz_tao_xin_pi*df_id.iloc[i]['最大值']/500+0.01))
    elif df_id.iloc[i]['是否套现'] == '是' and df_id.iloc[i]['新店旧店'] == '旧' and df_id.iloc[i]['波动程度']=='稳定':
        shop_shreshold.append(500*math.ceil(yz_tao_old_wen*df_id.iloc[i]['最大值']/500+0.01))
    elif df_id.iloc[i]['是否套现'] == '是' and df_id.iloc[i]['新店旧店'] == '旧' and df_id.iloc[i]['波动程度']=='普通':
        shop_shreshold.append(500*math.ceil(yz_tao_old_zho*df_id.iloc[i]['最大值']/500+0.01))
    elif df_id.iloc[i]['是否套现'] == '是' and df_id.iloc[i]['新店旧店'] == '旧' and df_id.iloc[i]['波动程度']=='波动':
        shop_shreshold.append(500*math.ceil(yz_tao_old_pi*df_id.iloc[i]['最大值']/500+0.01))
    else:
        continue

df_id['简单阈值'] = shop_shreshold


df_id.to_csv('总_商家套现阈值表.csv')    